In [1]:
import pandas as pd
import plotly.express as px

from statsmodels.formula.api import logit
from sklearn.preprocessing import OneHotEncoder

from DS_common.data_processing import pd_OHE

/home/dranik/.local/lib/python3.10/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


ModuleNotFoundError: No module named 'DS_common'

In [ ]:
train = pd.read_csv("train.csv", index_col = 0)
test = pd.read_csv("test.csv", index_col = 0)

In [ ]:
train.head()

# Data research

### Numeric variables

#### Age

In [ ]:
px.box(train, x = "Survived", y = "Age")

In [ ]:
train["q_age"] = pd.qcut(train['Age'],q = 30)

q_age_gb = train.groupby("q_age")
survived_by_age = pd.DataFrame({
    "Total count" : q_age_gb["Survived"].count(),
    "Survived count" : q_age_gb["Survived"].sum()
})

survived_by_age["Survived part"] = survived_by_age["Survived count"]/survived_by_age['Total count']
survived_by_age

### Fare

In [ ]:
px.box(train, x = "Survived", y = "Fare")

In [ ]:
train["q_fare"] = pd.qcut(train['Fare'],q = 30)

q_fare_gb = train.groupby("q_fare")
survived_by_fare = pd.DataFrame({
    "Total count" : q_fare_gb["Survived"].count(),
    "Survived count" : q_fare_gb["Survived"].sum()
})

survived_by_fare["Survived part"] = survived_by_fare["Survived count"]/survived_by_fare['Total count']
survived_by_fare

### Categorial variables

#### Pclass

In [ ]:
pd.crosstab(
    train["Pclass"],
    train["Survived"],
    normalize = "index"
)

#### Gender

In [ ]:
pd.crosstab(
    train["Sex"],
    train["Survived"],
    normalize = "index"
)

#### SibSp

In [ ]:
pd.concat([
    pd.crosstab(
        train["SibSp"],
        train["Survived"]
    ),
    pd.crosstab(
        train["SibSp"],
        train["Survived"],
        normalize = "index"
    )
], axis = 1)

#### Parch

In [ ]:
ct = pd.crosstab(
    train["Parch"], train["Survived"],
    normalize = "index"
)
ct["total count"] = train["Parch"].value_counts()

ct

#### Cabin

In [ ]:
train["Cabin_na_level"] = train["Cabin"].fillna("No data")

pd.crosstab(
    train["Cabin_na_level"],
    train["Survived"]
)

In [ ]:
train["Cabin_first_letter"] = train["Cabin"].str[0].fillna("No data")

ct = pd.crosstab(
    train["Cabin_first_letter"],
    train["Survived"],
    normalize = "index"
)

ct["total count"] = train["Cabin_first_letter"].value_counts()
ct

#### Embarked

In [ ]:
ct = pd.crosstab(
    train["Embarked"],
    train["Survived"],
    normalize = "index"
)
ct["total count"] = train["Embarked"].value_counts()
ct

#### Ticket

По самому по себе билету вряд ли, что-то получиться, потому немного преобразований над ним 

In [ ]:
train.loc[:,"Ticket"].isna().any()

In [ ]:
# отбросим номер (пропусков нет) потому все остальные наблюдения как без номера
train["Ticket code"] = train.loc[:,"Ticket"].str.extract("(.*) ").fillna("No code")
# Удалим точки так как предполагается, что например уровень SOTON/O.Q. то же самое, что и SOTON/OQ
train["Ticket code"] = train["Ticket code"].str.replace(".", "", regex = False)
# Все в верхний регистр так как предполагется, что, например, уровень SC/Paris то же самое, что и SC/PARIS
train["Ticket code"] = train["Ticket code"].str.upper()

In [ ]:
ct = pd.crosstab(
    train["Ticket code"], train["Survived"],
    normalize = "index" 
)
ct["total count"] = train["Ticket code"].value_counts()
ct

# Logit regression using all data

Helps to choose the best ways for work